In [1]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

import spacy
from tqdm import tqdm

import warnings 
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
train_df = pd.read_csv('train.tsv', delimiter='\t')
test_df = pd.read_csv('test.tsv', delimiter='\t')

In [3]:
train_df['review'] = train_df['review'].apply(lambda x: x.replace('<br />', ' '))
test_df['review'] = test_df['review'].apply(lambda x: x.replace('<br />', ' '))

In [4]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
scaler = MaxAbsScaler()
classifier = LogisticRegression()

In [5]:
model = make_pipeline(vectorizer, scaler, classifier)
model.fit(train_df['review'], train_df['is_positive']);

In [6]:
# Функция для оценки моделей:

def eval_model(model):
    pred = model.predict(test_df['review'])
    return accuracy_score(test_df['is_positive'], pred)

In [7]:
results = {}
results['minimum_preprocessing'] = eval_model(model)
print(f'Результат с минимальной предобработкой: {results["minimum_preprocessing"]}')

Результат с минимальной предобработкой: 0.90376


#### Задание 1.
Посмотрите на токены, если будут мусорные, добавьте их в стоп слова и обучите заново

In [8]:
vectorizer = CountVectorizer(ngram_range=(1, 2), max_df=0.4)
model = make_pipeline(vectorizer, scaler, classifier)
model.fit(train_df['review'], train_df['is_positive']);

In [9]:
results['exclude_stopwords'] = eval_model(model)
print(f'Результат без мусорных токенов: {results["exclude_stopwords"]}')

Результат без мусорных токенов: 0.90544


#### Задание 2. 
Проверьте, изменилось ли качество при лемматизации/и без неё

In [10]:
nlp = spacy.load('en_core_web_sm', disable=['parser'])
docs = nlp.pipe(train_df['review'])

In [11]:
lemmatized = []
for doc in tqdm(docs, total=len(train_df), position=0):
     lemmatized.append(' '.join(token.lemma_ for token in doc))

100%|████████████████████████████████████████████████████████████████████████████| 25000/25000 [07:07<00:00, 58.48it/s]


In [12]:
train_df['lemma'] = lemmatized

In [13]:
model.fit(train_df['lemma'], train_df['is_positive'])
results['lemmatization'] = eval_model(model)
print(f'Результат с применением лемматизации: {results["lemmatization"]}')

Результат с применением лемматизации: 0.86604


Видно, что результат стал значительно хуже.

#### Задание 3.
Замените все токены, которые принадлежат сущностям на их тег. Проверьте, изменилось ли качество после этого

In [14]:
docs = nlp.pipe(train_df['review'])

In [15]:
replaced = []
for doc in tqdm(docs, total=len(train_df), position=0):
    spam = []
    for token in doc:
        spam.append(token.ent_type_ if token.ent_type else token.text)
    replaced.append(' '.join(spam))

100%|████████████████████████████████████████████████████████████████████████████| 25000/25000 [07:13<00:00, 57.64it/s]


In [16]:
train_df['replace'] = replaced

In [17]:
model.fit(train_df['replace'], train_df['is_positive'])
results['replace_entities'] = eval_model(model)
print(f'Результат с заменой токенов, которые принадлежат сущностям на их тег: {results["replace_entities"]}')

Результат с заменой токенов, которые принадлежат сущностям на их тег: 0.90264


Лучше, чем после применения лемматизации, но чуть хуже, чем с минимальной предобработкой и избавлением от мусорных токенов.

#### Итог:

In [18]:
pd.DataFrame(results.values(), index=results.keys(), 
             columns=['accuracy']).sort_values('accuracy', ascending=False)

,accuracy
exclude_stopwords,0.90544
minimum_preprocessing,0.90376
replace_entities,0.90264
lemmatization,0.86604
